Convert test csv to mysql db


In [1]:
!pip install pandas mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 547.5 kB/s eta 0:00:0000:0100:01


In [13]:
import pandas as pd
import mysql.connector

# MySQL Connection Configuration
db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'password',
    'database': 'maid'
}

# CSV File Path
csv_file_path = 'test.csv'

def csv_to_mysql(csv_file_path, db_config):
    try:
        # Read CSV file into a DataFrame
        df = pd.read_csv(csv_file_path)

        # Connect to MySQL database
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Prepare SQL query to create table
        create_table_query = """
            CREATE TABLE IF NOT EXISTS testdata (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    battery_power FLOAT,
                    blue INT,
                    clock_speed FLOAT,
                    dual_sim INT,
                    fc FLOAT,
                    four_g INT,
                    int_memory INT,
                    m_dep FLOAT,
                    mobile_wt FLOAT,
                    n_cores INT,
                    pc FLOAT,
                    px_height FLOAT,
                    px_width FLOAT,
                    ram FLOAT,
                    sc_h FLOAT,
                    sc_w FLOAT,
                    talk_time INT,
                    three_g INT,
                    touch_screen INT,
                    wifi INT
            )
        """
        cursor.execute(create_table_query)

        # Iterate over each row in the DataFrame and insert into MySQL
        for _, row in df.iloc[:,1:].iterrows():
            # Prepare SQL query to insert data
            insert_query = """
            INSERT INTO testdata (battery_power, blue, clock_speed, dual_sim, fc,
                                    four_g, int_memory, m_dep, mobile_wt, n_cores,
                                    pc, px_height, px_width, ram, sc_h, sc_w,
                                    talk_time, three_g, touch_screen, wifi)
                VALUES (%s, %s, %s, %s, %s,
                        %s, %s, %s, %s, %s,
                        %s, %s, %s, %s, %s,
                        %s, %s, %s, %s, %s)
            """
        
            data = tuple(row)
            cursor.execute(insert_query, data)

        # Commit changes and close connection
        conn.commit()
        cursor.close()
        conn.close()
        print("Data inserted into the table successfully.")

    except mysql.connector.Error as e:
        print(f"An error occurred: {e}")
        # Rollback changes if an error occurs
        conn.rollback()
        cursor.close()
        conn.close()

# Call the function to convert CSV to MySQL
csv_to_mysql(csv_file_path, db_config)


Data inserted into the table successfully.


Test api for model predictions

In [ ]:
import requests

# URL of your Flask API endpoint
url = 'http://localhost:5000/predict'  

# Device specifications for prediction
device_specs = {
    'battery_power': 1500,
    'blue': 1,
    'clock_speed': 2.0,
    'dual_sim': 1,
    'fc': 5,
    'four_g': 1,
    'int_memory': 32,
    'm_dep': 0.3,
    'mobile_wt': 150,
    'n_cores': 8,
    'pc': 10,
    'px_height': 800.0,
    'px_width': 1600.0,
    'ram': 4096.0,
    'sc_h': 5,
    'sc_w': 4,
    'talk_time': 20,
    'three_g': 1,
    'touch_screen': 1,
    'wifi': 1
}

# Send POST request to the Flask API endpoint
response = requests.post(url, json=device_specs)

# Check if request was successful (status code 200)
if response.status_code == 200:
    # Extract predicted price from response
    predicted_price = response.json()['predicted_price']
    print(f'Predicted Price: {predicted_price}')
else:
    print(f'Error: {response.text}')
